<a href="https://colab.research.google.com/github/ma-anand/final/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("refresh the br df adfowser")

refresh the browser


In [ ]:
print("Hello")
print("hi")

Hello
hi
